In [4]:
import sys
import os
import timeit

current_dir = os.getcwd()

parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import *
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
import random

from Networks.DNN import PINN
from Networks.IA_DNN import Attention_PINN
from Networks.Fourier_DNN import Fourier_PINN
from Networks.AF_PINN import AF_PINN
from Networks.Efficient_KAN import KAN
from Networks.Cheby_KAN import ChebyKAN
from DataGenerator import DataGenerator
import pandas as pd

from torch.utils.data import TensorDataset, DataLoader

import time
from datetime import datetime

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_torch(42)

Using device: cuda:0


In [5]:
L = 600.0
h = 100.0
anneal_time= 60.0
D_A = 360.0  #6nm^2/s 360nm^2/min
k11,k12,k21 = 3.6e-3,3.6e-3,0.0
N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90.0,48.0,90.0,90.0,90.0,90.0

geom = [0,L]
time = [0,anneal_time]

##How I genertate data
generator = DataGenerator(geom,time)


from pde_system import compute_pde, compute_bc, compute_ic,relative_error
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
# from Utilitiy.residual_adaptive_resample import select_high_loss_points

true_path = './FDM/range[0,1]/'
file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]


In [6]:
hiddens = 4
neurons = 128

layer = [2] + [neurons]*hiddens + [6]
activation = torch.nn.functional.tanh
start_lr = 1e-3
end_lr = 1e-8

weight = [1, 1, 1, 1, 1]
from torch import Tensor
from pde_system import compute_pde, compute_bc, compute_ic
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
from Utilitiy.residual_adaptive_resample import select_high_loss_points

for mode in ['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']:  ##['PINN','IA_PINN','RES_PINN','RES_IA_PINN']
    loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
    torch.cuda.empty_cache()
    print(f'{mode}_{hiddens}x[{neurons}]')

    num_epochs = 5000
    resample_epochs = 10
    
    if mode == 'PINN':
        model = PINN(layer,activation,non_negative=True).to(device)
        optimizer = optim.Adam(model.parameters(), lr=start_lr)  
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * resample_epochs, eta_min=end_lr)
    elif mode == 'IA_PINN':
        model = Attention_PINN(layer,activation,non_negative=True).to(device)
        optimizer = optim.Adam(model.parameters(), lr=start_lr)  
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * resample_epochs, eta_min=end_lr)
    elif mode == 'Fourier_PINN':
        model = Fourier_PINN(layer,activation, non_negative=True, use_rff=True, rff_num_features=neurons, rff_sigma=1.0).to(device)
        optimizer = optim.Adam(model.parameters(), lr=start_lr)  
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * resample_epochs, eta_min=end_lr)
    elif mode == 'AF_PINN':
        model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features= neurons, rff_sigma=1.0).to(device)
        optimizer = optim.Adam(model.parameters(), lr=start_lr)  
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * resample_epochs, eta_min=end_lr)
    else:
        print("Man! What can I say! Wrong Mode!!!")
        break   

    points_types = ['domain', 'left_bc', 'right_bc', 'initial']

    coords_pde, coords_bc1, coords_bc2, coords_ic = [
        generator.grid_generator(N=2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
        for pt in points_types
    ]

    for idx in range(resample_epochs):
        print('第{}次重采样:----------------------'.format(idx+1))
        model.train()

        pbar = tqdm(range(num_epochs))
        
        for epoch in pbar:
            current_lr = optimizer.param_groups[0]['lr'] 
            def closure() -> Tensor:
                global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
                optimizer.zero_grad()

                # 计算损失时使用batch数据
                loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde)
                loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
                loss_bc1 = compute_bc(model, coords_bc1)
                loss_bc2 = compute_bc(model, coords_bc2)
                loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

                #Give a weight to initial loss term
                loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
                return loss

            optimizer.step(closure)
            # 如果使用scheduler（只在Adam阶段）
            if scheduler is not None:
                scheduler.step()
            
            loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2

            loss_log_total.append(loss_total.item())
            loss_log_pde.append(loss_pde.item())
            loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
            loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
            
            if epoch % 10 == 0:
                pbar.set_postfix({
                    'Loss': '{0:.3e}'.format(loss_total.item()),
                    'PDE': '{0:.3e}'.format(loss_pde.item()),
                    'BC': '{0:.3e},{1:.3e}'.format(loss_bc1.item(),loss_bc2.item()),
                    'IC': '{0:.3e},{1:.3e}'.format(loss_ic1.item(),loss_ic2.item()),
                    'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
                    'lr': '{0:.2e}'.format(current_lr),
                }) 

        model.eval()

        val_data = generator.grid_generator(N =10201,points_type= 'domain', nondim= True)
        result = model(val_data).data.cpu().numpy()
        relative_error(result,true_data)

        resample_nums = 0
        if idx % 2 ==0:
            resample_nums += 1
            start_time = timeit.default_timer()
            print(f"开始重采样: {resample_nums}")
            resample_pde = generator.LHS_generator(5000, points_type='domain', nondim=True)
            # resample_bc1 = generator.LHS_generator(1000, points_type='left_bc', nondim=True)
            # resample_bc2 = generator.LHS_generator(1000, points_type='right_bc', nondim=True)
            # resample_ic = generator.LHS_generator(5000, points_type='initial', nondim=True)

            add_pde = select_high_loss_points(model, resample_points= resample_pde, n_select= 250, loss_type='pde')
            # add_bc1 = select_high_loss_points(model, resample_points= resample_bc1, n_select= 200, loss_type='left_bc')
            # add_bc2 = select_high_loss_points(model, resample_points= resample_bc2, n_select= 200, loss_type='right_bc')
            # add_ic = select_high_loss_points(model, resample_points= resample_ic, n_select= 200, loss_type='initial')
        
            coords_pde = torch.cat((coords_pde, add_pde), dim=0)
            # coords_bc1 = torch.cat((coords_bc1, add_bc1), dim=0)
            # coords_bc2 = torch.cat((coords_bc2, add_bc2), dim=0)
            # coords_ic = torch.cat((coords_ic, add_ic), dim=0)
            
            # 将四个张量放在一个列表中并随机打乱
            shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

            # 解包到四个变量中
            coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
            
            elapsed = timeit.default_timer() - start_time
            print("结束重采样，用时: {0:.2f}s".format(elapsed),'---x采样特征：{0:.2f},{1:.2f}'.format(torch.min(add_pde[:,0]),torch.max(add_pde[:,0])),'---t采样特征:{0:.2f},{1:.2f}'.format(torch.min(add_pde[:,1]),torch.max(add_pde[:,1])))
        
    # 绘制结果
    plot_loss_curve(f'Re_[{neurons}]x{hiddens}_{mode}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
    
    model.eval()
    val_data = generator.grid_generator(N =10201,points_type= 'domain', nondim= True)
    result = model(val_data).data.cpu().numpy()
    
    plot_relative_concen(f'Re_[{neurons}]x{hiddens}_{mode}',result,data_save=True)

    
    true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]
    plot_3d_concen(f'Re_[{neurons}]x{hiddens}_{mode}', result, true_data)



PINN_4x[128]
第1次重采样:----------------------


100%|██████████| 5000/5000 [06:17<00:00, 13.24it/s, Loss=1.484e-02, PDE=2.865e-03, BC=1.974e-03,2.591e-05, IC=9.362e-03,6.087e-04, D*=5.99e-02,5.09e-03, lr=9.76e-04]


R^2 Error: {'Ni': 0.081, 'SiC': 0.019, 'C': 0.086, 'NiSi': 0.177, 'NiSi2': 0.21}
Inf Error: {'Ni': 0.078, 'SiC': 0.027, 'C': 0.17, 'NiSi': 0.274, 'NiSi2': 0.337}
Mean Error: (0.1146246652643447, 0.17711078535206218)
开始重采样: 1
结束重采样，用时: 80.15s ---x采样特征：0.00,0.29 ---t采样特征:0.00,0.99
第2次重采样:----------------------


100%|██████████| 5000/5000 [06:15<00:00, 13.30it/s, Loss=9.604e-03, PDE=2.693e-03, BC=2.375e-04,4.044e-06, IC=6.232e-03,4.376e-04, D*=6.00e-02,4.17e-03, lr=9.05e-04]


R^2 Error: {'Ni': 0.062, 'SiC': 0.022, 'C': 0.069, 'NiSi': 0.155, 'NiSi2': 0.163}
Inf Error: {'Ni': 0.056, 'SiC': 0.023, 'C': 0.131, 'NiSi': 0.244, 'NiSi2': 0.268}
Mean Error: (0.0941278809172095, 0.14436714363470554)
第3次重采样:----------------------


...

R^2 Error: {'Ni': 0.041, 'SiC': 0.005, 'C': 0.026, 'NiSi': 0.062, 'NiSi2': 0.053}
Inf Error: {'Ni': 0.042, 'SiC': 0.009, 'C': 0.047, 'NiSi': 0.087, 'NiSi2': 0.1}
Mean Error: (0.03722791758393436, 0.056968573403461654)
开始重采样: 1
结束重采样，用时: 161.61s ---x采样特征：0.00,0.28 ---t采样特征:0.00,0.56
第10次重采样:----------------------


100%|██████████| 5000/5000 [17:16<00:00,  4.83it/s, Loss=1.715e-03, PDE=1.743e-04, BC=3.292e-06,3.785e-07, IC=1.445e-03,9.214e-05, D*=6.00e-02,1.86e-03, lr=1.01e-08]


R^2 Error: {'Ni': 0.038, 'SiC': 0.004, 'C': 0.035, 'NiSi': 0.065, 'NiSi2': 0.061}
Inf Error: {'Ni': 0.039, 'SiC': 0.008, 'C': 0.057, 'NiSi': 0.088, 'NiSi2': 0.1}
Mean Error: (0.040612924680272396, 0.05833032163078703)
Fourier_PINN_4x[128]
第1次重采样:----------------------


 24%|██▍       | 1214/5000 [02:08<06:41,  9.44it/s, Loss=3.410e-02, PDE=8.271e-03, BC=4.339e-03,4.694e-05, IC=2.055e-02,8.964e-04, D*=5.99e-02,4.65e-03, lr=9.99e-04]


KeyboardInterrupt: 